In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

### Data Collection

In [ ]:
import requests

# URLs of the files
train_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module5/exercise/module5_exercise_train.csv'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module5/exercise/module5_exercise_test.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_data_url, 'module5_exercise_train.csv')
download_file(test_data_url, 'module5_exercise_test.csv')

In [ ]:
df_train =  pd.read_csv("module5_exercise_train.csv", sep=",")
df_test =  pd.read_csv("module5_exercise_test.csv", sep=",")

In [ ]:
df_train.isna().sum()

### Data analysis

In [ ]:
#### Make a complete analysis on data preprocessing
# Inconsistencies
# Duplicates (data.duplicated().sum())
# Missing values (data.isnull().sum())
# Categorical
# Outliers
# Feature Engineering
# Feature Selection and/or Dimensionality Reduction

In [ ]:
def plot_feature_over_time(df, feature, date_id_start, date_id_end):
    df_filtered = df[(df['date'] >= date_id_start) & (df['date'] <= date_id_end)]
    
    if feature not in df_filtered.columns:
        print(f"Feature '{feature}' not found in the DataFrame.")
        return
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df_filtered['date'], df_filtered[feature], label=feature, linestyle='-')
    plt.xlabel('Date')
    plt.ylabel(feature)
    plt.title(f'{feature} from {date_id_start} to {date_id_end}')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()



In [ ]:
plot_feature_over_time(data, 'electricity_demand', '2015-01-01', '2021-09-07')

In [ ]:
plot_feature_over_time(data, 'humidity', '2016-06-01', '2016-12-01')

### Data Preprocessing Evaluation Strategy

In [ ]:
# Provide a complete data preprocessing transformations
df_test.isna().sum()

In [ ]:
#Conversion vitesse du vent en float km/h
def to_m_s(value):
    if isinstance(value, str):
        value = value.strip()
        if 'm/s' in value:
            num = float(value.replace(' m/s', '').strip())
            return num * 3.6
        elif 'km/h' in value:
            num = float(value.replace(' km/h', '').strip())
            return num
    return None  # si ce n'est pas une string ou format inconnu


df_train.head()


In [ ]:
# 1. Handle Inconsistencies
def handle_inconsistencies(X_train, y_train, X_val=None):
    X_train = X_train.copy()
    y_train = y_train.copy()
    X_train['wind_speed'] = X_train['wind_speed'].apply(to_m_s)
    if X_val is not None:
        X_val = X_val.copy()
        X_val['wind_speed'] = X_val['wind_speed'].apply(to_m_s)
        return X_train, y_train, X_val
    else:
        return X_train, y_train
    


def handle_duplicates(X_train, y_train, X_val=None):
    """
    Supprime les doublons exacts dans X_train (et garde la première occurrence).
    """
    X_train = X_train.copy()
    y_train = y_train.copy()
    if X_val is not None:
        X_val = X_val.copy()

    # Supprimer les doublons
    duplicates_mask = X_train.duplicated(keep='first')
    X_train_no_duplicates = X_train[~duplicates_mask]
    y_train_no_duplicates = y_train.loc[X_train_no_duplicates.index]

    if X_val is not None:
        return X_train_no_duplicates, y_train_no_duplicates, X_val
    else:
        return X_train_no_duplicates, y_train_no_duplicates

from sklearn.impute import SimpleImputer

# 3. Handling Missing Values
def handle_missing_values(X_train, y_train = None, X_val=None):
    numerical_cols = X_train.select_dtypes(include=['number']).columns
    cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
    imputer = SimpleImputer(strategy='mean')
    X_train[numerical_cols] = imputer.fit_transform(X_train[numerical_cols])

    # Imputation des catégorielles avec le mode
    cat_imputer = SimpleImputer(strategy='most_frequent')
    X_train[cat_cols] = cat_imputer.fit_transform(X_train[cat_cols])

    y_train = y_train.copy()
    imputery = SimpleImputer(strategy='mean')
    y_train = imputery.fit_transform(y_train.to_frame()).ravel() 

    if X_val is not None:
        X_val[numerical_cols] = imputer.transform(X_val[numerical_cols])
        X_val[cat_cols] = cat_imputer.transform(X_val[cat_cols])  
        return X_train.copy(), y_train, X_val.copy()
    else:

        return X_train.copy(), y_train

# 4. Handling Categorical Values
def handle_categorical(X_train, y_train, X_val=None):
    X_train = X_train.copy()
    X_train = pd.get_dummies(X_train, drop_first=False)  
    if X_val is not None:
        X_val = X_val.copy()
        X_val = pd.get_dummies(X_val, drop_first=False)
        return X_train.copy(), X_val.copy()
    else:
        return X_train.copy()

# 5. Handling Outliers


# 6. Feature Engineering
def feature_engineering(X_train, y_train, X_val=None):
    #Feature Engineering
    # 1. Convertir en datetime
    X_train['date'] = pd.to_datetime(X_train['date'], format='%Y-%m-%d')

    # 2. Créer une colonne binaire : 1 = week-end, 0 = jour de semaine
    X_train['is_weekend'] = X_train['date'].dt.dayofweek >= 5

    # Optionnel : transformer en int (0 ou 1)
    X_train['is_weekend'] = X_train['is_weekend'].astype(int)

    X_train.drop('date', axis=1, inplace=True)
   
    if X_val is not None:

        X_val['date'] = pd.to_datetime(X_val['date'], format='%Y-%m-%d')  
        X_val['is_weekend'] = X_val['date'].dt.dayofweek >= 5
        X_val['is_weekend'] = X_val['is_weekend'].astype(int)
        X_val.drop('date', axis=1, inplace=True)
        return X_train.copy(), y_train, X_val.copy()
    else:
        return X_train.copy(), y_train

# 7. Feature Selection and Dimensionality Reduction
def feature_selection(X_train, y_train, X_val=None):
    selected_columns = ['humidity', 'temperature_station1',
       'temperature_station2', 'temperature_station3', 'temperature_station4',
       'temperature_station5', 'temperature_station6', 'temperature_station7',
       'temperature_station8', 'temperature_station9', 'temperature_station10']
    if X_val is not None:
        return X_train[selected_columns], X_val[selected_columns]
    else:
        return X_train[selected_columns]

In [ ]:


def handle_outliers(X_train, y_train=None, X_val=None):
    X_train = X_train.copy()
    if y_train is not None:
        y_train = y_train.copy()
    if X_val is not None:
        X_val = X_val.copy()
    
    numeric_cols = X_train.select_dtypes(include='number').columns

    # Boucle sur chaque colonne numérique
    for col in numeric_cols:
        Q1 = X_train[col].quantile(0.25)
        Q3 = X_train[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        # Remplacer les outliers par NaN
        X_train[col] = X_train[col].mask((X_train[col] < lower) | (X_train[col] > upper))

    # Même traitement pour y si c'est une Series
    if y_train is not None and isinstance(y_train, pd.Series):
        Q1 = y_train.quantile(0.25)
        Q3 = y_train.quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        y_train = y_train.mask((y_train < lower) | (y_train > upper))

    if X_val is not None:
        return X_train, y_train, X_val
    else:
        return X_train, y_train


In [ ]:
numerical_cols = df_train.select_dtypes(include='number').columns.drop('electricity_demand')
cat_cols = df_train.select_dtypes(include='object').columns

X = df_train.drop('electricity_demand', axis=1)
X_test = df_test.copy()
#X_cat_cols = df_train[cat_cols]
y = df_train['electricity_demand']

X, y = handle_outliers(X, y)


In [ ]:
X 

In [ ]:
def evaluate_pipeline(X, y, n_splits=5):

    ### call transformations here, if there is no learning and no need to be crossval
    X, y = handle_inconsistencies(X, y)
    # X, y = handle_duplicates(X, y)
    X  = handle_missing_values(X, y)
    # X_train = handle_categorical(X, y)
    X, y = handle_outliers(X, y)
    # X, y = feature_engineering(XX, y)
    X = feature_selection(X, y)
    
    model = LinearRegression()
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    train_scores = []
    val_scores = []
    
    for fold, (train_index, val_index) in enumerate(tscv.split(X)):
        print(f"Processing fold {fold + 1}/{n_splits}...")
        
        # Split data into train and validation sets
        X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
        y_train, y_val = y.iloc[train_index].copy(), y.iloc[val_index].copy()

        ### call transformations here, if there is learning
        # X_train, y_train, X_val = handle_inconsistencies(X_train, y_train, X_val)
        X_train, y_train, X_val = handle_duplicates(X_train, y_train, X_val)
        # X_train, X_val = handle_missing_values(X_train, y_train, X_val)
        X_train, X_val = handle_categorical(X_train, y_train, X_val)
        # X_train, y_train, X_val = handle_outliers(X_train, y_train, X_val)
        X_train, y_train, X_val = feature_engineering(X_train, y_train, X_val)
        # X_train, X_val = feature_selection(X_train, y_train, X_val)
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on training set
        y_train_pred = model.predict(X_train)
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_scores.append(train_mse)
        
        # Predict on validation set
        y_val_pred = model.predict(X_val)
        val_mse = mean_squared_error(y_val, y_val_pred)
        val_scores.append(val_mse)
        
        print(f"Fold {fold + 1} Train MSE: {train_mse:.4f}, Validation MSE: {val_mse:.4f}")
    
    # Compute mean, max, and min values for train and validation MSE
    mean_train_mse = np.mean(train_scores)
    max_train_mse = np.max(train_scores)
    min_train_mse = np.min(train_scores)
    
    mean_val_mse = np.mean(val_scores)
    max_val_mse = np.max(val_scores)
    min_val_mse = np.min(val_scores)
    
    # Print results
    print("\nTrain MSE:")
    print(f"Mean: {mean_train_mse:.4f}, Max: {max_train_mse:.4f}, Min: {min_train_mse:.4f}")
    
    print("\nValidation MSE:")
    print(f"Mean: {mean_val_mse:.4f}, Max: {max_val_mse:.4f}, Min: {min_val_mse:.4f}")
    
    return mean_val_mse  # Return mean validation MSE as the overall score

In [ ]:
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

# Run the evaluation
evaluate_pipeline(X, y)

In [ ]:
model = LinearRegression()
model.fit(X, y)

# --- 3️⃣ Prédire sur le test ---
y_pred = model.predict(X)

# --- 4️⃣ Calculer la MSE ---
mse = mean_squared_error(y, y_pred)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

ridge = Ridge(alpha=1.0)  # alpha = paramètre de régularisation
ridge.fit(X, y)

# Prédire et évaluer
y_pred = ridge.predict(X)
mse = mean_squared_error(y, y_pred)

In [ ]:
mse

In [ ]:
import seaborn as sns

cov_matrix = X.cov()  # matrice de covariance

# Affichage heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cov_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Heatmap de covariance")
plt.show()

In [ ]:
cov_matrix


In [ ]:
mse

In [ ]:
evaluate_pipeline(X, y)

### Generating Submission File

In [ ]:
# Train and submit your results

In [ ]:
# Prepare X_train and y_train from your data
df_train =  pd.read_csv("module5_exercise_train.csv", sep=",")

X_train = df_train.drop(columns=['electricity_demand'], axis=1)
y_train = df_train['electricity_demand']

X_test =  pd.read_csv("module5_exercise_test.csv", sep=",")

In [ ]:

X_train, y_train, X_test = feature_engineering(X_train, y_train, X_test)

X_train, y_train, X_test = handle_inconsistencies(X_train, y_train, X_test)
X_train, y_train, X_test = handle_outliers(X_train, y_train, X_test)

X_train, y_train, X_test = handle_duplicates(X_train, y_train, X_test)
X_train, y_train, X_test = handle_missing_values(X_train, y_train, X_test)
X_train, X_test = handle_categorical(X_train, y_train, X_test)
    #X_train, X_test = feature_selection(X_train, y_train, X_test)

In [ ]:
def train_and_predict_to_submit(X_train, y_train, X_test):
    model = LinearRegression()
    
    X_train, y_train, X_test = feature_engineering(X_train, y_train, X_test)

    X_train, y_train, X_test = handle_inconsistencies(X_train, y_train, X_test)
    X_train, y_train, X_test = handle_outliers(X_train, y_train, X_test)

    X_train, y_train, X_test = handle_duplicates(X_train, y_train, X_test)
    X_train, y_train, X_test = handle_missing_values(X_train, y_train, X_test)
    X_train, X_test = handle_categorical(X_train, y_train, X_test)
    #X_train, X_test = feature_selection(X_train, y_train, X_test)

    # Train the model on the entire training set
    print(f"Training model on entire dataset of shape: {X_train.shape}")
    model.fit(X_train, y_train)
    
    # Predict on the test set
    print(f"Predicting on test dataset of shape: {X_test.shape}")
    y_test_pred = model.predict(X_test)
    
    return y_test_pred

In [ ]:
# Call serve_model to train and predict
y_test_pred = train_and_predict_to_submit(X_train, y_train, X_test)

In [ ]:
# Generating Submission File
submission = pd.DataFrame({
    'date': X_test['date']
    'electricity_demand': y_test_pred
})

# Save the submission file
submission.to_csv('submission.csv', index=False, sep=',')
print("Submission file saved as 'submission.csv'.")